# Hands-on: Deploying Question Answering with BERT

Pre-trained language representations have been shown to improve many downstream NLP tasks such as question answering, and natural language inference. Devlin, Jacob, et al proposed BERT [1] (Bidirectional Encoder Representations from Transformers), which fine-tunes deep bidirectional representations on a wide range of tasks with minimal task-specific parameters, and obtained state- of-the-art results.

After finishing training QA with BERT (the previous notebook "QA_Training.ipydb"), let us load a trained model to perform inference on the SQuAD dataset

### A quick overview: an example from SQuAD dataset is like below:

    (2, 
    '56be4db0acb8001400a502ee', 
    'Where did Super Bowl 50 take place?', 

    'Super Bowl 50 was an American football game to determine the champion of the National 
    Football League (NFL) for the 2015 season. The American Football Conference (AFC) 
    champion Denver Broncos defeated the National Football Conference (NFC) champion 
    Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played 
    on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, 
    California. As this was the 50th Super Bowl, the league emphasized the "golden 
    anniversary" with various gold-themed initiatives, as well as temporarily suspending 
    the tradition of naming each Super Bowl game with Roman numerals (under which the 
    game would have been known as "Super Bowl L"), so that the logo could prominently 
    feature the Arabic numerals 50.', 

    ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium 
    in the San Francisco Bay Area at Santa Clara, California."], 

    [403, 355, 355])

## Deploy on SageMaker

1. Preparing functions for inference 
2. Saving the model parameters
3. Building a docker container with dependencies installed
4. Launching a serving end-point with SageMaker SDK

### 1. Preparing functions for inference

Two functions: 
1. model_fn() to load model parameters
2. transform_fn() to run model inference given an input

In [1]:
# %%writefile code/serve.py
import collections, json, logging, warnings
import multiprocessing as mp
from functools import partial

import gluonnlp as nlp
import mxnet as mx
from mxnet.gluon import Block, nn
from bert.data.qa import preprocess_dataset, SQuADTransform
import bert_qa_evaluate


class BertForQA(Block):
    """Model for SQuAD task with BERT.
    The model feeds token ids and token type ids into BERT to get the
    pooled BERT sequence representation, then apply a Dense layer for QA task.
    Parameters
    ----------
    bert: BERTModel
        Bidirectional encoder with transformer.
    prefix : str or None
        See document of `mx.gluon.Block`.
    params : ParameterDict or None
        See document of `mx.gluon.Block`.
    """

    def __init__(self, bert, prefix=None, params=None):
        super(BertForQA, self).__init__(prefix=prefix, params=params)
        self.bert = bert
        with self.name_scope():
            self.span_classifier = nn.Dense(units=2, flatten=False)

    def forward(self, inputs, token_types, valid_length=None):  # pylint: disable=arguments-differ
        """Generate the unnormalized score for the given the input sequences.
        Parameters
        ----------
        inputs : NDArray, shape (batch_size, seq_length)
            Input words for the sequences.
        token_types : NDArray, shape (batch_size, seq_length)
            Token types for the sequences, used to indicate whether the word belongs to the
            first sentence or the second one.
        valid_length : NDArray or None, shape (batch_size,)
            Valid length of the sequence. This is used to mask the padded tokens.
        Returns
        -------
        outputs : NDArray
            Shape (batch_size, seq_length, 2)
        """
        bert_output = self.bert(inputs, token_types, valid_length)
        output = self.span_classifier(bert_output)
        return output


def model_fn(model_dir = "", params_path = "bert_qa-7eb11865.params"):
    """
    Load the gluon model. Called once when hosting service starts.
    :param: model_dir The directory where model files are stored.
    :return: a Gluon model, and the vocabulary
    """
    bert_model, vocab = nlp.model.get_model('bert_12_768_12',
                                        dataset_name='book_corpus_wiki_en_uncased',
                                        use_classifier=False,
                                        use_decoder=False,
                                        use_pooler=False,
                                        pretrained=False)
    net = BertForQA(bert_model)
    if len(model_dir) > 0:
        params_path = model_dir + "/" +params_path
    net.load_parameters(params_path, ctx=mx.cpu())
    
    tokenizer = nlp.data.BERTTokenizer(vocab,  lower=True)
    transform = SQuADTransform(tokenizer, is_pad=False, is_training=False, do_lookup=False)
    return net, vocab, transform


def transform_fn(model, input_data, input_content_type=None, output_content_type=None):
    """
    Transform a request using the Gluon model. Called once per request.
    :param model: The Gluon model and the vocab
    :param dataset: The request payload
    
        Example:
        ## (example_id, [question, content], ques_cont_token_types, valid_length, _, _)

    :param input_content_type: The request content type, assume json
    :param output_content_type: The (desired) response content type, assume json
    :return: response payload and content type.
    """
    net, vocab, squadTransform = model
    data = json.loads(input_data)
    test_examples_tuples = bert_qa_evaluate._test_example_transform(data)
    test_dataset = mx.gluon.data.SimpleDataset(test_examples_tuples)
    all_results = bert_qa_evaluate.get_all_results(net, vocab, squadTransform, test_dataset, ctx=mx.cpu())
    all_predictions = collections.defaultdict(list)
    data_transform = test_dataset.transform(squadTransform._transform)
    for features in data_transform:
        f_id = features[0].example_id
        results = all_results[f_id]
        prediction, nbest = bert_qa_evaluate.predict(
            features=features,
            results=results,
            tokenizer=nlp.data.BERTBasicTokenizer(vocab))        
        nbest_prediction = [] 
        for i in range(3):
            nbest_prediction.append('%.2f%% \t %s'%(nbest[i][1] * 100, nbest[i][0]))
        all_predictions[f_id] = nbest_prediction
    response_body = json.dumps(all_predictions)
    return response_body, output_content_type


### 2. Saving the model parameters

In [2]:
## save model parameters and vocabulary in a zip file

import tarfile
with tarfile.open("model.tar.gz", "w:gz") as tar:
#     tar.add("Dockerfile")
    tar.add("code/serve.py")
    tar.add("bert/data/qa.py")
    tar.add("bert_qa_evaluate.py")
    tar.add("bert_qa-7eb11865.params")
    tar.add("vocab.json")

In [3]:
## test
my_test_example_0 = ('Which NFL team represented the AFC at Super Bowl 50?',
 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.')

my_test_example_1 = ('Where did Super Bowl 50 take place?',
 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.')

my_test_examples = (my_test_example_0, my_test_example_1)

# mymodel = model_fn(params_path = "bert_qa-7eb11865.params")
# transform_fn(mymodel, my_test_examples)

### 3. Building a docker container with dependencies installed

Let's prepare a docker container with all the dependencies required for model inference. Here we build a docker container based on the SageMaker MXNet inference container, and you can find the list of all available inference containers at https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html

Here we use local mode for demonstration purpose. To deploy on actual instances, you need to login into AWS elastic container registry (ECR) service, and push the container to ECR. 

```
docker build -t $YOUR_EDR_DOCKER_TAG . -f Dockerfile
$(aws ecr get-login --no-include-email --region $YOUR_REGION)
docker push $YOUR_EDR_DOCKER_TAG
```

In [4]:
%%writefile Dockerfile

ARG REGION
FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/mxnet-inference:1.4.1-gpu-py3

RUN pip install --upgrade --user --pre 'mxnet-mkl' 'https://github.com/dmlc/gluon-nlp/tarball/v0.9.x'

RUN pip list | grep mxnet

COPY *.py /opt/ml/model/code/
COPY bert/data/qa.py /opt/ml/model/code/bert/data/
COPY bert/bert_qa_evaluate.py /opt/ml/model/code/bert/

Overwriting Dockerfile


In [5]:
!export REGION=$(wget -qO- http://169.254.169.254/latest/meta-data/placement/availability-zone) &&\
 docker build --no-cache --build-arg REGION=${REGION::-1} -t my-docker:inference . -f Dockerfile

Sending build context to Docker daemon  843.5MB
Step 1/7 : ARG REGION
Step 2/7 : FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/mxnet-inference:1.4.1-gpu-py3
 ---> d9dd4dcfe0c2
Step 3/7 : RUN pip install --upgrade --user --pre 'mxnet-mkl' 'https://github.com/dmlc/gluon-nlp/tarball/v0.9.x'
 ---> Running in 74f0ee0ac65f
ERROR: mxnet-mkl 1.6.0b20191025 has requirement numpy<2.0.0,>1.16.0, but you'll have numpy 1.14.6 which is incompatible.
  Running setup.py install for gluonnlp: started
    Running setup.py install for gluonnlp: finished with status 'done'
You should consider upgrading via the 'pip install --upgrade pip' command.
Removing intermediate container 74f0ee0ac65f
 ---> bc3bcf04918d
Step 4/7 : RUN pip list | grep mxnet
 ---> Running in 85ce4ab37b38
You should consider upgrading via the 'pip install --upgrade pip' command.
keras-mxnet                       2.2.4.1       
mxnet-cu100mkl                    1.4.1         
mxnet-mkl                         1.6.0b20191025
mxnet-mode

### 4. Launching a serving end-point with SageMaker SDK

We create a MXNet model which can be deployed later, by specifying the docker image, and entry point for the inference code. If serve.py does not work, use dummy_hosting_module.py for debugging purpose. 

In [6]:
import sagemaker
from sagemaker.mxnet.model import MXNetModel
sagemaker_model = MXNetModel(model_data='file:///home/ec2-user/SageMaker/ako2020-bert/tutorial/model.tar.gz',
                             image='my-docker:inference', # docker images
                             role=sagemaker.get_execution_role(), 
                             py_version='py3',            # python version
                             entry_point='serve.py',
                             source_dir='.')

We use 'local' mode to test our deployment code, where the inference happens on the current instance.
If you are ready to deploy the model on a new instance, change the `instance_type` argument to values such as `ml.c4.xlarge`.

Here we use 'local' mode for testing, for real instances use c5.2xlarge, p2.xlarge, etc.

In [7]:
predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type='local')

Attaching to tmp_f5n_fa7_algo-1-jf0uq_1
algo-1-jf0uq_1  | 2020-01-15 00:54:59,352 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
algo-1-jf0uq_1  | MMS Home: /usr/local/lib/python3.6/site-packages
algo-1-jf0uq_1  | Current directory: /
algo-1-jf0uq_1  | Temp directory: /home/model-server/tmp
algo-1-jf0uq_1  | Number of GPUs: 0
algo-1-jf0uq_1  | Number of CPUs: 8
algo-1-jf0uq_1  | Max heap size: 13646 M
algo-1-jf0uq_1  | Python executable: /usr/local/bin/python3.6
algo-1-jf0uq_1  | Config file: /etc/sagemaker-mms.properties
algo-1-jf0uq_1  | Inference address: http://0.0.0.0:8080
algo-1-jf0uq_1  | Management address: http://127.0.0.1:8081
algo-1-jf0uq_1  | Model Store: /.sagemaker/mms/models
algo-1-jf0uq_1  | Initial Models: ALL
algo-1-jf0uq_1  | Log dir: /logs
algo-1-jf0uq_1  | Metrics dir: /logs
algo-1-jf0uq_1  | Netty threads: 0
algo-1-jf0uq_1  | Netty client threads: 0
algo-1-jf0uq_1  | Default workers per model: 8
algo-1-jf0uq_1  | Blacklist Regex: N/A
algo-1-jf0uq_1  | Maximum Re

algo-1-jf0uq_1  | 2020-01-15 00:54:59,926 [INFO ] pool-1-thread-9 ACCESS_LOG - /172.18.0.1:60880 "GET /ping HTTP/1.1" 200 36
!

In [8]:
output = predictor.predict(my_test_examples)  

algo-1-jf0uq_1  | 2020-01-15 00:55:02,760 [INFO ] W-9001-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 3045
algo-1-jf0uq_1  | 2020-01-15 00:55:02,784 [INFO ] W-9007-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 3073
algo-1-jf0uq_1  | 2020-01-15 00:55:02,788 [INFO ] W-9005-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 3095
algo-1-jf0uq_1  | 2020-01-15 00:55:02,796 [INFO ] W-9001-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Vocab file is not found. Downloading.
algo-1-jf0uq_1  | 2020-01-15 00:55:02,796 [INFO ] W-9001-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Downloading /root/.mxnet/models/1579049702.7962046book_corpus_wiki_en_uncased-a6607397.zip from https://apache-mxnet.s3-accelerate.dualstack.amazonaws.com/gluon/dataset/vocab/book_corpus_wiki_en_uncased-a6607397.zip...
algo-1-jf0uq_1  | 2020-01-15 00:55:02,796 [INFO ] W-9003-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time

In [15]:
print("\nPrediction output: \n\n")

for k in output.keys():
    print('{}\n\n'.format(output[k]))


Prediction output: 


['99.36% \t Denver Broncos', '0.23% \t The American Football Conference (AFC) champion Denver Broncos', '0.20% \t Broncos']


["25.86% \t Levi's Stadium in the San Francisco Bay Area at Santa Clara, California", "23.11% \t Levi's Stadium", '17.88% \t San Francisco Bay Area at Santa Clara, California']




### Clean Up

Remove the endpoint after we are done. 

In [16]:
predictor.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)
